### 1. Motivation
#### What is your dataset?
We used three datasets related to Austin Animal Center:
- Austin_Animal_Center_Intakes_20250419.csv: records of animals entering the shelter, including intake dates and animal types.
- Austin_Animal_Center_Outcomes_20250419.csv: outcome records of animals leaving the shelter (e.g., adoption, euthanasia).
- shelter_geocoded_locations.csv: a manually geocoded dataset that maps official shelter addresses in Austin to their corresponding latitude and longitude coordinates, based on data from the Austin Animal Center's official shelter list.
- neighbour.json: downloaded from https://data.austintexas.gov/.

#### Why did you choose this/these particular dataset(s)?
The intake dataset offers a rich, time-stamped record of real-world animal shelter operations. With the help of manually geocoded shelter addresses, we were able to conduct meaningful spatial analysis. This multi-angle approach allows for an accessible and engaging visual story.

#### What was your goal for the end user's experience?
We want users to understand the patterns behind stray animal intake: when animals enter shelters most frequently, and which shelter regions are under more pressure. The visualizations should guide users from temporal overview to regional insights.

### 2. Basic stats
#### Data Cleaning & Preprocessing Steps
- Merged intake & outcome tables: According to Animal ID.Keeping duplicates for repeat visits.
- Date parsing: Converted intake_datetime to proper datetime format.
- Shelter matching: Linked each record to a shelter location based on the shelter_name or address field.
- Geolocation: Used shelter_geocoded_locations.csv to attach coordinates to each shelter site.
- Neighborhood mapping: Mapped coordinates to official Austin neighborhood polygons using spatial joins.
- Time aggregation: Grouped intake counts by month and by shelter region.
- Outliers: Removed records with missing animal type or unmatchable shelter data.

#### Dataset Statistics
- Intakes dataset rows: 126,234
- Outcomes dataset rows: 124,513
- Unique animal types: 5 (Dogs, Cats, Birds, Others, Small Mammals)
- Mapped shelter coordinates: Successfully matched to all listed shelters
- Time span: October 2013 to April 2025
- Number of mapped shelters: 12

### 3. Data Analysis

### 4. Genre
#### Which genre of data story did you use?
We used a Martini Glass narrative structure: The stem is a focused, linear presentation (time series → heatmap animation),followed by an open exploration body (static choropleth map for spatial analysis and comparison).

#### Visual Narrative tools used
- Graphical highlighting (e.g., saturation of heatmap points over time)
- Progressive reveal (the animated heatmap adds time dimension interactively)
- Visual grouping (e.g., color mapping in the choropleth for comparing regions)

#### Narrative Structure tools used
- Author-driven sequence at the start (clear framing of the problem through time trend)
- Reader-driven interaction in choropleth map and animation slider
- Multi-messaging via annotation blocks and map legends

###  5. Visualizations
We created three key visualizations to support our story:
- Monthly Intake Time Series
→ Highlights long-term trends and seasonal fluctuations
- Animated Heatmap of Found Locations
→ Shows how animal sightings spread and shift over time

- Choropleth Map by Neighborhood
→ Allows spatial comparison of intake burden across city districts

- Shelter Coverage & Intakes Map 
→ Shows where intake hotspots align—or don’t—with current 5 km shelter coverage.

- Intake → Outcome Flow Sankey 
→ Summarises how each intake source splits into final outcomes

### 6. Discussion
#### What went well?
The time series plot clearly visualizes trend and seasonality.The animated heatmap engages viewers while providing intuitive geographic context.The choropleth map makes it easy to spot disparities across neighborhoods.

#### What is still missing? What could be improved? Why?
We lost about 55% of geocoded addresses, which reduced the spatial accuracy. An interactive dashboard combining filters (e.g., by animal type, intake condition) would offer deeper insights. Labeling and overlays on maps could enhance interpretability for first-time viewers.

### 7. Contributions